In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

In [54]:
# 1.1 Importer les données Train et Test

train_clean = pd.read_csv("dev/machineLearning/deepLearning/data/Module3/train_clean.csv")

test_clean = pd.read_csv("dev/machineLearning/deepLearning/data/Module3/test_clean.csv")


In [55]:
# 1.2 Depuis le Dataframe train, charger les features d'apprentissage dans un array numpy X_alltrain, et les labels (données à prévoir) dans un array numpy y_alltrain

X_alltrain = train_clean.values[:, 2:]
y_alltrain = train_clean.values[:, 1]

In [56]:
# 1.3 Séparer les features et les labels en deux parties (train et dev), en attribuant 10% des exemples aux données de dev. Afficher les nombres de lignes et de colonnes pour les 4 arrays.

X_train, X_dev, y_train, y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=42)

print("Dimensions de X_train :", X_train.shape)
print("Dimensions de y_train :", y_train.shape)
print("Dimensions de X_dev :", X_dev.shape)
print("Dimensions de y_dev :", y_dev.shape)

Dimensions de X_train : (801, 7)
Dimensions de y_train : (801,)
Dimensions de X_dev : (90, 7)
Dimensions de y_dev : (90,)


In [57]:
# 1.4 Afficher les 10 premières lignes de features et les 10 premiers labels.

print("10 premières lignes de X_train :\n", X_train[:10, :])

print("10 premiers éléments de y_train :\n", y_train[:10])


10 premières lignes de X_train :
 [[3 1 0 2 0 0 4]
 [3 0 0 3 0 0 2]
 [1 1 3 3 0 1 1]
 [3 1 2 0 2 1 1]
 [3 1 0 0 0 1 1]
 [3 1 1 0 0 1 1]
 [1 1 2 3 0 0 1]
 [3 1 2 1 0 1 1]
 [2 0 2 1 0 1 3]
 [3 1 1 0 0 1 1]]
10 premiers éléments de y_train :
 [1 0 0 0 0 0 1 1 1 0]


In [72]:
# 2.1 Définir et instancier une classe Titanic Model avec les caractéristiques suivantes :
#  - Deux couches cachées de 50 neurones.
#  - Deux classes en sortie : Survivant ou non
#  - Des fonctions d'activation RELU
#  - Un dropout paramétrable pour les 2 couches cachées

class TitanicModel(nn.Module):
    def __init__(self, dropout_rate):
        super(TitanicModel, self).__init__()

        self.fc1 = nn.Linear(7, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(50, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, inputs):
        x = self.fc1(inputs)
        x = self.relu1(inputs)
        x = self.dropout1(inputs)
        x = self.fc2(inputs)
        x = self.relu2(inputs)
        x = self.dropout2(inputs)
        x = self.fc3(inputs)
        x = self.softmax(inputs)

        output = self.output_layer(x)
        return output

titanic = TitanicModel(dropout_rate=0.4)

In [67]:
# 2.2 Définir des paramètres de nombre d'epochs (50) et de learning_rate (0.01)

num_epochs = 50
learning_rate = 0.01

In [68]:
# 2.3 Définir la taille du minibatch à 50. En déduire le nombre de boucles pour chaque epoch.

batch_size = 50
num_batches = len(X_train) // batch_size
num_steps_per_epoch = num_batches + 1 if len(X_train) % batch_size != 0 else num_batches

In [69]:
# 2.4 Définir une fonction de coût de type CrossEntropy

criterion = F.cross_entropy

In [70]:
# 2.5. Définir un optimizer de type Adam, sans oublier le learning rate

optimizer = optim.Adam(titanic.parameters(), learning_rate)

In [71]:
# 3.1 Exécuter l'apprentissage du modèle.
#  - Créer une boucle sur les epochs, qui contient elle-même une boucle sur les minibatchs.
#  - À chaque nouvelle itération sur les epochs, mélanger les données avec la méthode shuffle.
#  - Tous les 5 epochs afficher la valeur de la fonction de cout

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_dev = torch.tensor(X_dev, dtype=torch.float32)
y_dev = torch.tensor(y_dev, dtype=torch.long)

for epoch in range(num_epochs):
    indices = torch.randperm(len(X_train))
    X_train_shuffled = X_train[indices]
    y_train_shuffled = y_train[indices]

    for i in range(0, len(X_train), batch_size):
        X_batch = X_train_shuffled[i:i+batch_size]
        y_batch = y_train_shuffled[i:i+batch_size]

        outputs = titanic(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass et optimisation des poids
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Affichage de la perte tous les 5 epochs
    if (epoch + 1) % 5 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

/tmp/ipykernel_11530/1023758026.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
/tmp/ipykernel_11530/1023758026.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long)
/tmp/ipykernel_11530/1023758026.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_dev = torch.tensor(X_dev, dtype=torch.float32)
/tmp/ipykernel_11530/1023758026.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() o

RuntimeError: mat1 and mat2 shapes cannot be multiplied (50x7 and 50x50)

In [73]:
# 3.2 Calculer la précision de la prévision sur les données dev

titanic.eval()

with torch.no_grad():
    dev_inputs = torch.Tensor(X_dev)
    dev_labels = torch.Tensor(y_dev)
    dev_outputs = titanic(dev_inputs)
    _, predicted = torch.max(dev_outputs.data, 1)
    total = dev_labels.size(0)
    correct = (predicted == dev_labels).sum().item()
    accuracy = correct / total

print('Accuracy on dev set: {:.2f}%'.format(accuracy * 100))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (90x7 and 50x50)

In [ ]:
# 3.3 Calculer prévisions sur les données de tests



In [ ]:
# 3.4 Générer le fichier résultat et l'envoyer sur kaggle. Quel est votre score et votre classement ?

In [ ]:
# 3.5 Exécuter une cross-validation dans une boucle pour trouver les meilleures valeurs de learning rate, de keep_prob et de nombre d'epochs.
